<a href="https://colab.research.google.com/github/Ikhadija-5/AMMI-NLP-LABS/blob/main/Khadija_Iddrisu_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [52]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [53]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [54]:
print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [55]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    # # ## FILL CODE   
    for sentence in data:
      for word in sentence:
        if word not in vocab or vocab[word] <= mincount:
          sentence[sentence.index(word)] = '<unk>'
      data_with_unk.append(sentence)

    return data_with_unk



In [56]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [57]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
    #     ## FILL CODE
    #     # dict can be indexed by tuples
    #     # store in the same dict all the ngrams
    #     # by using the context as a key and the word as a value

        for i in range(len(sentence)):
            temp = sentence[i:n+i]
            for j in range(len(temp)):
                counts[tuple(temp[:j])][temp[j]] += 1


    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # # Build the probabilities from the counts
    # # Be careful with how you normalize!

    #     ## FILL CODE
    for context in counts.keys():
        s = sum(counts[context].values())
        for w in counts[context]:
            prob[context][w] = 1.0 * counts[context][w] / s

    #     # for context in counts.keys():
    return prob


In [58]:
# RUN TO BUILD NGRAM MODEL
n = 3
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  3


In [59]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE
    
    if model[tuple(context)][w] != 0:
      prob_ = model[tuple(context)][w] 
    else:
       prob_ = 0.4 * get_prob(model, context[1:], w)
    return prob_


In [60]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    score = count = 0
    for words in data:
        context = words[:n-1]
        for w in words[n-1:]:
            score += np.log(get_prob(model, context, w))
            # context.pop(0)
            context.append(w)
            count += 1
    score = - score/count

    return score

In [61]:
# COMPUTE PERPLEXITY ON VALIDATION SET
print("The perplexity is", perplexity(model, valid_data, n=3))

The perplexity is 11.170018150367495


In [62]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    

    ## FILL CODE
    # code a recursive function over context
    # to find the longest available ngram
    if sum(model[tuple(context)].values()) > 0:
      words_and_probs = model[tuple(context)]
    else:
      words_and_probs = get_proba_distrib(model, context[1:])
    return words_and_probs


In [63]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    ## FILL CODE
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    #   np.random.choice(x, 1, p = y)
    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    while True :
        world_dict = get_proba_distrib(model, sentence)
        p = np.random.choice(list(world_dict.keys()), 1, p = list(world_dict.values()))[0]
        sentence.append(p)
        if p == "</s>": break
    return sentence


In [64]:
# GENERATE A SENTENCE FROM THE MODEL
print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'this', 'world', 'is', 'full', 'of', 'life,', 'before', 'he', 'was', '<unk>', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.